# VacationPy

In [1]:
##### *** Import Dependencies *** #####

import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os
from pprint import pprint

##### *** API Configuration *** #####

## Import API Key
from config import api_key_google

## Google Maps API URL
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
##### *** Initialize Needed Variables *** #####

## Input File Location 
input_data_file = "city_weather.csv"


In [2]:
### Store Part I Results into a Data Frame

weather_data = pd.read_csv(input_data_file) 
weather_data.head()

length = len(weather_data)
length

598

### Humidity Heatmap

In [3]:
### Configure gmaps w/ API Key
gmaps.configure(api_key=api_key_google)

### Store City Locations & Humidity (our "heat" value) as Floats
locations = weather_data[['Lat', 'Long']].astype(float)
weights = weather_data['Humidity'].astype(float)
weights_max = weather_data['Humidity'].max()

### Create & Map Humidity "Heatmap" layer
fig = gmaps.figure()
humidity_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=weights_max,
                                 point_radius = 1)
fig.add_layer(humidity_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create New Dataframe Fitting Personalized Weather Criteria

In [4]:
### Narrow the Number of Cities to Include Your Ideal Conditions
weather_ideal = weather_data[(weather_data.Max_Temp <= 93.0) &
                             (weather_data.Cloudiness <= 15.0) &
                             (weather_data.Humidity <= 20.0) &
                             (weather_data.Wind_Speed <= 5.0)]

weather_ideal_length = len(weather_ideal) # there are 8 cities within ideal conditions

### Drop NULL Values
weather_ideal = weather_ideal.dropna() # there were no NULL values to drop, still 8 cities

##### *** Save Dataframe as .csv *** #####
weather_ideal.to_csv('weather_ideal.csv', encoding='utf-8', index=False) # Note to avoid issues, use encoding="utf-8"
weather_ideal


,Date,City_ID,City_Name,Country,Lat,Long,Max_Temp,Cloudiness,Humidity,Wind_Speed
85,2020-02-25,1253237.0,Verāval,IN,20.90,70.37,90.7,0.0,15.0,4.8
157,2020-02-25,1274304.0,Chhindwāra,IN,22.07,78.93,78.6,15.0,17.0,3.4
206,2020-02-25,72181.0,Marzuq,YE,14.40,46.47,78.5,0.0,14.0,1.0
212,2020-02-25,5301388.0,Lake Havasu City,US,34.48,-114.32,57.2,1.0,19.0,2.1
310,2020-02-25,1274040.0,Chittaurgarh,IN,24.88,74.63,83.6,0.0,17.0,3.6
460,2020-02-25,1266397.0,Kodīnar,IN,20.79,70.70,90.7,0.0,15.0,4.8
462,2020-02-25,104515.0,Makkah al Mukarramah,SA,21.43,39.83,65.6,0.0,15.0,2.9
473,2020-02-25,2447513.0,Arlit,NE,18.74,7.39,54.3,1.0,17.0,1.2
485,2020-02-25,2449893.0,Tessalit,ML,20.20,1.01,61.1,0.0,14.0,4.4
519,2020-02-25,2450173.0,Taoudenni,ML,22.68,-3.98,58.7,0.0,18.0,2.9


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

### Perform API Calls to Generate Hotels Dataframe

In [5]:
##### *** Initialize Variable Objects Needed for API Calls *** #####
hotel_df = weather_ideal[["City_Name","Country","Lat", "Long"]]
hotel_df["Hotel Name"] = ""

## Set Up a Parameters Dictionary

parameters = {
#     "location": ideal_locations,
    "radius": 5000,
    "type": "lodging",
#     "fields": google_fields,
    "key": api_key_google}

##### *** Perform API Calls *** #####

for index, row in hotel_df.iterrows():
    Latitude = row["Lat"]
    Longitude = row["Long"]
    parameters["location"] = f"{Latitude},{Longitude}"
    
    ## API Request
    hotels_data = requests.get(url, params=parameters)
    google_response = hotels_data.json()
    try:
        name = google_response["results"][0]['name']
        hotel_df.loc[index,"Hotel Name"] = name
    except(KeyError, IndexError):
        print("not good")

print(hotel_df)
    


C:\Users\MC\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\MC\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


not good
not good
not good
                City_Name Country    Lat    Long  \
85                Verāval      IN  20.90   70.37   
157            Chhindwāra      IN  22.07   78.93   
206                Marzuq      YE  14.40   46.47   
212      Lake Havasu City      US  34.48 -114.32   
310          Chittaurgarh      IN  24.88   74.63   
460               Kodīnar      IN  20.79   70.70   
462  Makkah al Mukarramah      SA  21.43   39.83   
473                 Arlit      NE  18.74    7.39   
485              Tessalit      ML  20.20    1.01   
519             Taoudenni      ML  22.68   -3.98   

                        Hotel Name  
85               Lords Inn Somnath  
157         STARiHOTELS Chhindwara  
206                                 
212                     Heat Hotel  
310  Palette - The Elegance Resort  
460    Hotel Krishna Park, Kodinar  
462        Al Shohada Hotel MAKKAH  
473           Hôtel Telwa Bungalow  
485                                 
519                           

In [6]:

hotel_df=hotel_df.rename(columns={"City_Name": "City"})

# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]


In [7]:
# Add marker layer ontop of heat map


### Create & Map Humidity "Heatmap" layer
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
                              
fig.add_layer(marker_layer)
fig



Figure(layout=FigureLayout(height='420px'))